In [19]:
import torch
import numpy
import pennylane as qml
import pennylane.numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt

In [24]:
def Cliff2():
    """
    Random 2-qubit Clifford circuit.

    Arguments:
        -nodes (np.ndarray): 
    
    Returns:
        -null
    """
    weights = np.random.randint(2, size=(2, 10))
    
    return qml.matrix(qml.RandomLayers(weights=weights,wires=[0,1])).numpy()

In [60]:
N_QUBITS = 2*2
DEPTH = 3

random_layers = []
for t in range(DEPTH):
        layer = []
        for x in range(0,N_QUBITS,2):
                layer.append(Cliff2())
        random_layers.append(layer)


dev = qml.device("default.qubit", wires=N_QUBITS)

@qml.qnode(dev)
def circuit(theta):
    """
    Quantum circuit with random entangling Clifford layers and disentangling layers.
    
    Arguments:
        -theta (np.ndarray): Binary matrix representing the positions of projections. (N_QUBITS, DEPTH)
    
    Returns:
        -Average Von Neumann entropy (float32): Average of 2-qubit Von Neumann entropies over all neighbors.
    """

    theta = theta.T
    DEPTH,N_QUBITS = np.shape(theta)

    for t in range(DEPTH):
        layer = random_layers[t]
        if t%2==0:
            for x in range(0,N_QUBITS,2):
                brick = layer[int(x/2)]
                qml.QubitUnitary(brick,wires=[x,x+1])
        elif t%2==1:
            for x in range(1,N_QUBITS-2,2):
                brick = layer[int((x-1)/2)]
                qml.QubitUnitary(brick,wires=[x,x+1])
            brick = layer[-1]
            qml.QubitUnitary(brick,wires=[N_QUBITS-1,0])
            
        projections = theta[t]
        for x in range(N_QUBITS):
            if projections[x]==1:
                qml.Projector(state=[0],wires=[x])
            
    entropies = []
    for x in range(N_QUBITS-1):
        entropies.append(qml.vn_entropy(wires=[x,x+1]))
    entropies.append(qml.vn_entropy(wires=[N_QUBITS-1,0]))
        
    return entropies

In [61]:
random_layers[0][1]

array([[ 0.14875996-0.52570305j, -0.03440755-0.37360505j,
         0.0629826 +0.68387947j, -0.08126794+0.28719288j],
       [-0.27046856+0.12621802j,  0.47639351-0.49467884j,
         0.26025496-0.27024428j, -0.49508938+0.23104054j],
       [-0.47639351-0.49467884j,  0.27046856+0.12621802j,
        -0.49508938-0.23104054j,  0.26025496+0.27024428j],
       [-0.03440755+0.37360505j,  0.14875996+0.52570305j,
         0.08126794+0.28719288j, -0.0629826 +0.68387947j]])

In [68]:
theta = np.random.randint(2, size=(N_QUBITS,DEPTH))
print(circuit(theta))
drawer = qml.draw(circuit)

print(drawer(theta))

[-0.0, 3.540321126392621e-15, 2.351930950646267e-15, -0.0]
0: ─╭U(M0)──|0⟩⟨0|────────╭U(M3)─╭U(M4)───|0⟩⟨0|─────────┤ ╭vnentropy                      
1: ─╰U(M0)──|0⟩⟨0|─╭U(M2)─│──────╰U(M4)──────────────────┤ ╰vnentropy ╭vnentropy           
2: ─╭U(M1)──|0⟩⟨0|─╰U(M2)─│──────────────╭U(M5)──────────┤            ╰vnentropy ╭vnentropy
3: ─╰U(M1)──|0⟩⟨0|────────╰U(M3)──|0⟩⟨0|─╰U(M5)───|0⟩⟨0|─┤                       ╰vnentropy

  ╭vnentropy
  │         
  │         
  ╰vnentropy

M0 = 
[[ 0.20671135-0.41507207j -0.12387677-0.13597415j -0.85544338+0.09774769j
   0.09774769-0.01608029j]
 [-0.09774769-0.01608029j  0.43742952-0.7416154j   0.07378964+0.16849188j
   0.20671135+0.41507207j]
 [-0.43742952-0.7416154j   0.09774769-0.01608029j  0.20671135-0.41507207j
   0.07378964-0.16849188j]
 [-0.12387677+0.13597415j  0.20671135+0.41507207j -0.09774769-0.01608029j
   0.85544338+0.09774769j]]
M1 = 
[[ 0.14875996-0.52570305j -0.03440755-0.37360505j  0.0629826 +0.68387947j
  -0.08126794+0.28719288j]
 

In [69]:
def RandomFlip(theta,K):
    """
    Randomly flip K entries of a binary matrix theta.

    Arguments:
        -theta (np.ndarray):
        -K (int): 

    Returns:
        -flipped (np.ndarray):

    """
    
    x,y = np.shape(theta)
    N = int(x*y)
    arr = np.array([0] * (N-K) + [1] * K)
    np.random.shuffle(arr)
    arr = arr.reshape(np.shape(theta))

    flipped = (theta + arr) % 2
    
    return flipped

In [88]:
class Disentangler():
    """
    Reinforcement learning environment for the disentangler.

    """
    
    def __init__(self):
        self.action_space = [5]
        self.observation_space = []
        self.state = np.zeros((N_QUBITS,DEPTH))
        self.moves = 10

    def step(self, action):
        self.state = RandomFlip(self.state, action)
        self.moves += -1

        entropies = circuit(self.state)
        entropy = sum(entropies)/len(entropies)
        
        if entropy < 1e-16:
            reward = 10
        elif entropy > 1e-16:
            reward = -1

        if self.moves <= 0:
            done = True
        else:
            done = False

        info = {}
        
        return self.state, reward, done, info
    
    def reset(self):
        self.state = np.zeros((N_QUBITS,DEPTH))

        return self.state

In [89]:
env = Disentangler()

In [90]:
episodes = 50
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        action = env.action_space[0]
        n_state, reward, done, info = env.step(action)
        score += reward
    
    print('Episode:{} Score:{}'.format(episode, score))


Episode:1 Score:12
Episode:2 Score:-1
Episode:3 Score:-1
Episode:4 Score:-1
Episode:5 Score:-1
Episode:6 Score:-1
Episode:7 Score:-1
Episode:8 Score:-1
Episode:9 Score:-1
Episode:10 Score:-1
Episode:11 Score:-1
Episode:12 Score:-1
Episode:13 Score:-1
Episode:14 Score:-1
Episode:15 Score:-1
Episode:16 Score:-1
Episode:17 Score:-1
Episode:18 Score:-1
Episode:19 Score:-1
Episode:20 Score:-1
Episode:21 Score:-1
Episode:22 Score:-1
Episode:23 Score:-1
Episode:24 Score:-1
Episode:25 Score:-1
Episode:26 Score:-1
Episode:27 Score:-1
Episode:28 Score:-1
Episode:29 Score:-1
Episode:30 Score:-1
Episode:31 Score:-1
Episode:32 Score:-1
Episode:33 Score:-1
Episode:34 Score:-1
Episode:35 Score:-1
Episode:36 Score:-1
Episode:37 Score:-1
Episode:38 Score:-1
Episode:39 Score:-1
Episode:40 Score:-1
Episode:41 Score:-1
Episode:42 Score:-1
Episode:43 Score:-1
Episode:44 Score:-1
Episode:45 Score:-1
Episode:46 Score:-1
Episode:47 Score:-1
Episode:48 Score:-1
Episode:49 Score:-1
Episode:50 Score:-1


In [91]:
n_state

tensor([[1., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.]], requires_grad=True)